In [6]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import OSM, get_provider

import geopandas as gpd
import numpy as np

In [ ]:
gdf = gpd.read_file('MarketingArea.geojson')

# gdf.crs = 'EPSG:4326'
# gdf = gdf.to_crs(epsg=3857)

# gdf = gdf[['id', 'name', 'name:en', 'geometry']]
# gdf['id'] = gdf['id'].apply(lambda x: int(x[9:]))

gdf.reset_index(inplace=True)

gdf.columns = ['area_id', 'area_name', 'area_geometry']
gdf

In [ ]:
vendor_locs = pd.read_csv('vendor_locs.csv')
vendor_locs.head()

In [ ]:
## COUNTY
gdf = gpd.read_file('Iran-County-Mrz.geojson')

gdf = gdf[['name', 'name:en', 'geometry']]
gdf.reset_index(inplace=True)

gdf.columns = ['county_id', 'county_name_fa', 'county_name_en', 'county_geometry']
gdf.head()

In [ ]:
gdf = gpd.read_file('MarketingArea.geojson')

gdf.reset_index(inplace=True)

gdf.columns = ['area_id', 'area_name', 'area_geometry']
gdf.head()

In [ ]:
from shapely import wkt

areas = pd.read_excel('MarketingAreas.xlsx')

areas = areas.pipe(gpd.GeoDataFrame)
areas.columns = ['City', 'AreaID', 'AreaTitle', 'AreaTitle_English', 'SuperArea', 'geometry']

areas['geometry'] = areas['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(areas)
# gdf = gpd.GeoDataFrame(areas, crs='epsg:4326')
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(epsg=3857)

In [ ]:
from shapely import wkt

areas = pd.read_csv('Areas.csv')

def tryParseWKT(x):
    try:
        return wkt.loads(x)
    except:
        return None

areas['geometry'] = areas['new_polygon'].apply(lambda x: tryParseWKT(x))
# areas['new_geometry'] = areas['new_geometry'].apply(lambda x: tryParseWKT(x))

gdf = gpd.GeoDataFrame(areas[areas['geometry'].notna()], crs='epsg:4326')

gdf = gdf.to_crs(epsg=3857)
# gdf = gdf.to_crs(epsg=4326)

In [ ]:
gdf.head()

In [94]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def in_polygon(lat, lng):
    point = Point(lng, lat)
    for i, row in gdf.iterrows():
        polygon = row['area_geometry']
        if point.within(polygon):
            return row

In [95]:
vendor_locs[['area_id', 'area_name', 'area_geometry']] = vendor_locs.apply(lambda x: in_polygon(x.latitude, x.longitude), axis = 1)

In [ ]:
vendor_locs[vendor_locs.county_id.notna()].fillna('no_area', inplace=True)

In [107]:
vendor_locs.to_csv('vendor_areas.csv', index=False)

In [ ]:
def to_decimal(S):
    S = str(S)
    try:
        if '°' not in S:
            return float(S)
        S = S.split('"')[0]
        S = float(S.split('°')[0]) + float(S.split('°')[1].split("'")[0])/60 + float(S.split('°')[1].split("'")[1])/3600
        return S
    except:
        return np.nan

df = pd.read_excel("RefahLocations_Damoun_211030.xlsx")

df['Lat'] = df['t'].apply(lambda x: to_decimal(x))
df['Long'] = df['g'].transform(lambda x: to_decimal(x))

In [ ]:
S = "59°34'09626"
float(S.split('°')[0]) + float(S.split('°')[1].split("'")[0])/60 + float(S.split('°')[1].split("'")[1])/3600

In [ ]:
df.head()

In [ ]:
# df = pd.read_excel("Chains Locs.xlsx")
# x = df.groupby('Store').apply(lambda x: in_polygon(x.lat, x.lng)).reset_index()
df = df.merge(x, on='Store', how='left')

df.rename(columns={0:'AreaID'}, inplace=True)

df = df.merge(areas, on='AreaID', how='left')

In [ ]:
df

In [ ]:
df.to_excel('Chains Locs_Area.xlsx', index=False)

## MAPs

In [12]:
import json
from bokeh.models import GeoJSONDataSource

def get_geodatasource(df):
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

In [13]:
def merc_from_arrays(lats, lons):
    r_major = 6378137.000
    x = r_major * np.radians(lons)
    scale = x/lons
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + lats * (np.pi/180.0)/2.0)) * scale
    return (x, y)

### Areas with Ranks

In [ ]:
radius = 2750
df = pd.read_csv(f'../farm/notebooks/kdtree/coverage_score_{radius}.csv')
df

In [ ]:
plot_this = gdf.merge(df, left_on=['CityID', 'AreaID'], right_on=['customer_city_id', 'customer_area_id'], how='inner')
plot_this['score'].fillna(0, inplace=True)
plot_this

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Greys256, Turbo256
import colorcet as cc
from bokeh.models.mappers import LinearColorMapper


color_mapper = LinearColorMapper(palette=cc.fire)

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("City", "@customer_city"),
                     ("CityID", "@customer_city_id"),
                     ("AreaTitle", "@customer_area"),
                     ("Score", "@score")])

p.add_tile(tile_provider)

p.patches('xs','ys', source=get_geodatasource(plot_this),
          fill_alpha=0.8, fill_color={'field': 'score', 'transform': color_mapper},
          line_color="black", line_width=0.6)

# longitude, latitude = merc_from_arrays(np.array(dark_stores.latitude), np.array(dark_stores.longitude))
# source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, ID=np.array(dark_stores['id']), title=np.array(dark_stores.title)))
# p.circle(x='longitude', y='latitude', radius=500, fill_color='Black', fill_alpha=0.5, source=source)

# source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, ID=np.array(dark_stores['id']), title=np.array(dark_stores.title)))
# p.circle(x='longitude', y='latitude', radius=3000, fill_color=None, fill_alpha=0.2, source=source)



output_file(f'coverage_{radius}.html')

show(p)

### Genral Maps

In [ ]:
df['colorcode'] = np.where(df['Vendor']=='canbo', '#3366ff', np.nan)
df['colorcode'] = np.where(df['Vendor']=='hayat', '#ff9933', df['colorcode'])
df['colorcode'] = np.where(df['Vendor']=='refah', '#33cc33', df['colorcode'])
df['colorcode'] = np.where(df['Vendor']=='hyper family', '#cc3399', df['colorcode'])
df['colorcode'] = np.where(df['Vendor']=='canbo zooket', '#ff0000', df['colorcode'])

In [ ]:
df = pd.read_excel("damun.xlsx")
v = pd.read_excel('updateVendors.xlsx')

In [ ]:
gdf.head(2)

In [25]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.mappers import LinearColorMapper
# import colorcet as cc

# color_mapper = LinearColorMapper(palette=cc.isolum)

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[('Area', "@name")])

p.add_tile(tile_provider)

# for pr, c in zip(clusters['Priority'].unique(), ['#3366ff', '#ff9933', '#33cc33', '#cc3399']):
#     df = clusters[clusters['Priority']==pr]

#     longitude, latitude = merc_from_arrays(np.array(df['clat'].astype(float)), np.array(df['clong'].astype(float)))

#     source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, Cluster=np.array(df['Cluster']), 
#                                         Priority=np.array(df['Priority'])))
#     p.scatter(x='longitude', y='latitude', radius=50, fill_alpha=0.9, 
#               fill_color=c, source=source)

#     source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, Cluster=np.array(df['Cluster']), 
#                                         Priority=np.array(df['Priority']), radius=np.array(df['Min Distance'])))
#     p.scatter(x='longitude', y='latitude', radius='radius', fill_alpha=0.8,
#               fill_color=c, source=source)

# for bl, c in zip(vendors['BusinessLine'].unique(), ['#BDC3C7', '#707B7C', '#1C2833']):
#     df = vendors[vendors['BusinessLine']==bl]

#     longitude, latitude = merc_from_arrays(np.array(df['latitude'].astype(float)), np.array(df['longitude'].astype(float)))

#     source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, VendorID=np.array(df['Vendor']), 
#                                         BusinessLine=np.array(df['BusinessLine'])))
#     p.scatter(x='longitude', y='latitude', radius=100, fill_alpha=0.9, 
#               fill_color=c, source=source)

# longitude, latitude = merc_from_arrays(np.array(df['latitude'].astype(float)), np.array(df['longitude'].astype(float)))
# source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude,
#                                     VendorID=np.array(df['Store']),
#                                    AreaTitle=np.array(df['AreaTitle'])))
# p.scatter(x='longitude', y='latitude', radius=10, fill_alpha=0.7,
#           fill_color='black', source=source)
# p.scatter(x='longitude', y='latitude', radius=500, fill_alpha=0.3,
#           fill_color='black', source=source)

# longitude, latitude = merc_from_arrays(np.array(centorids['clat'].astype(float)), np.array(centorids['clong'].astype(float)))

# source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, Cluster=np.array(centorids['c'])))
# p.scatter(x='longitude', y='latitude', radius=1000, fill_alpha=0.3,
#           fill_color='Blue', source=source)



########### FOODAREAS
p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.8, color = 'grey',
          line_color="black", line_width=0.2)

# output_file('Clusters expansion.html')

show(p)

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', 
           tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom")

p.add_tile(tile_provider)

p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.5, color = 'grey',
          line_color="black", line_width=0.2)

# output_file('lat_long.html')

show(p)

### Food Area Coverage

In [ ]:
import pymssql

query = """select MarketingAreaName, u.latitude, u.longitude, count(o.id), avg(o.sub_total)
    from Orders o
    join UsersAddresses u on u.id = o.user_address_id
    join marketing_db.dbo.Vendors K on K.id = o.vendor_id
    where status_id = 123
      and K.SuperTypeID=4
      and K.CityID = 1
      and o.created_at >= '2021-07-19'
    group by MarketingAreaName, u.latitude, u.longitude;"""

conn = pymssql.connect(server='warehouse.snappfood.ir:1410', user='zooket', password='?t!ytN9JW39<3[zj', database='marketing_db')
orders_cor = pd.read_sql(query, conn)
orders_cor.columns = ['SuperArea', 'latitude', 'longitude', 'OrderCount', 'AVB']
conn.close()

In [ ]:
from haversine import haversine

radius = 3.5
orders_cor['dist'] = orders_cor.apply(lambda x: haversine((x.latitude, x.longitude), (35.764198303222656,51.42612838745117)), axis=1)

In [ ]:
avb = orders_cor[orders_cor['dist']<radius+0.05]['AVB'].mean()
dist = (orders_cor[orders_cor['dist']<radius+0.05]['OrderCount'].sum()*100) / orders_cor['OrderCount'].sum()

In [ ]:
avb, dist

In [ ]:
orders_cor = orders_cor.reset_index()

In [ ]:
x = orders_cor.groupby('index').apply(lambda x: in_polygon(x.latitude, x.longitude)).reset_index()

In [ ]:
x.columns = ['index', 'ID']

recent_orders = recent_orders.merge(x, on='index', how='left')
recent_orders = recent_orders.merge(areas, on='ID', how='left')

In [ ]:
teh = recent_orders[recent_orders['ID'].notna()].groupby(['City', 'Title_English']).agg({'OrderCount':'sum'}).reset_index()
teh = teh.merge(recent_orders[['City', 'Title_English', 'geometry']].drop_duplicates(), how='left', on=['City', 'Title_English'])
teh['OrderPerc'] = teh['OrderCount']*100/sum(teh['OrderCount'])

In [ ]:
gdf = gpd.GeoDataFrame(teh, crs='epsg:4326')

gdf = gdf.to_crs(epsg=3857)

In [ ]:
gdf.head()

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("OrderPerc", "@OrderPerc"),
                    ("OrderCount", "@OrderCount"),
                    ("SnappFoodArea", "@Title_English"),])
p.add_tile(tile_provider)

p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.3, color = 'grey',
          line_color="black", line_width=0.6)

output_file('FoodArea_Coverage.html')

show(p)

### Query

In [ ]:
import pymssql

conn = pymssql.connect(server='87.247.185.67:1433', user='CoreTeam', password='clwkehr2o8349h', database='ZooketDW')
q = """select V.id as 'Store', V.title, V.latitude as 'Lat', V.longitude as 'Long', V.status
        from Vendors V
            join VendorTypes VT on vendor_type_id = VT.id
        where vendor_super_type_id = 4
          and V.status <> 'Suspend';"""

vendors = pd.read_sql(q, conn)
conn.close()

In [ ]:
df = df.merge(vendors, on='Store', how='left')

In [ ]:
df.tail()

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral5

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("VendorID", "@VendorID"),
                   ("Title", "@title"),
                    ("Status", "@status")])
p.add_tile(tile_provider)

for this_status, name, color in zip(df.status.unique(), df.status.unique(), Spectral5):
    this_df = df[df.status==this_status]
    longitude, latitude = merc_from_arrays(np.array(this_df.Lat), np.array(this_df.Long))
    source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, 
                                        status=np.array(this_df.status), 
                                        VendorID=np.array(this_df.Store),
                                        title=np.array(this_df.title)))
    p.circle(x='longitude', y='latitude', radius=500, fill_alpha=1, source=source, color=color, legend_label=name)

    
p.legend.location = "top_right"
p.legend.click_policy="hide"

# longitude, latitude = merc_from_arrays(np.array(df.Lat), np.array(df.Long))
# source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(df.Store), Comp=np.repeat('Refah', len(latitude))))
# p.scatter(x='longitude', y='latitude', radius=100, fill_color='#83206B', fill_alpha=1, source=source)
# p.scatter(x='longitude', y='latitude', radius=2700, fill_color='#83206B', fill_alpha=0.3, source=source)

# p.patches('xs','ys', source=get_geodatasource(gdf),
#           fill_alpha=0.3, color = 'grey',
#           line_color="black", line_width=0.6)

output_file('Nazanin.html')

show(p)

In [ ]:
########## GANDOM CANBO ZOOKET

from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("Company", "@Comp"),
                    ("StoreID", "@StoreID"),
                    ("SnappFoodArea", "@AreaTitle_English"),])
p.add_tile(tile_provider)

longitude, latitude = merc_from_arrays(np.array(Vendors.lat), np.array(Vendors.lng))
source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(Vendors.Store), Comp=np.repeat('Zooket', len(latitude))))
p.circle(x='longitude', y='latitude', radius=100, fill_color='#F1E864', fill_alpha=0.9, source=source)

longitude, latitude = merc_from_arrays(np.array(canbo.lat), np.array(canbo.lng))
source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(canbo.Store), Comp=np.repeat('Canbo', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=100, fill_color='#83206B', fill_alpha=0.9, source=source)

longitude, latitude = merc_from_arrays(np.array(gandom.lat), np.array(gandom.lng))
source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(gandom.StoreID), Comp=np.repeat('Gandom', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=100, fill_color='#32CD32', fill_alpha=0.9, source=source)

p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.3, fill_color = '#d3d3d3',
          line_color="black", line_width=1)

output_file('Zooket_Canbo_Gandom_Areas.html')

show(p)

### Vendor Status

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

query = """select Vendors.id, Vendors.title, Vendors.status, Cities.title as `city`,Vendors.latitude,Vendors.longitude
            from zoodfood_db.Vendors
                     join zoodfood_db.VendorTypes on Vendors.vendor_type_id = VendorTypes.id
                     JOIN zoodfood_db.Cities on Vendors.city_id = Cities.id
            where vendor_super_type_id = 4
              and Vendors.status in ('ACTIVE');"""

cnx = mysql.connector.connect(user='mohammad_hossein', password='clwkehr2o8349h', host='db.snappfood.ir', port='3306')
Vendors = pd.read_sql(query, cnx)
cnx.close()

In [ ]:
Vendors.head()

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import inferno

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("VendorID", "@id"),
                    ("VendorTitle", "@title"),
                    ("City", "@city"),
                    ("Status", "@status"),])
p.add_tile(tile_provider)
Vendors= data.copy()

Vendors.longitude, Vendors.latitude = merc_from_arrays(np.array(Vendors.latitude), np.array(Vendors.longitude))
colormap = {}
for v, color in zip(Vendors.status.unique(), inferno(Vendors.status.nunique())):
    colormap[v] = color

colormap['ACTIVE'] = 'Green'
Vendors['colors'] = [colormap[x] for x in Vendors['status']]
source = ColumnDataSource(Vendors)

p.circle(x='longitude', y='latitude', color='colors', fill_alpha=0.7, radius=200, legend_field='status', source=source)
p.legend.location = "top_left"

output_file('Zooket_Vendor_Status.html')

# show(p)

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom")
p.add_tile(tile_provider)
Vendors= data.copy()

Vendors.longitude, Vendors.latitude = merc_from_arrays(np.array(Vendors.latitude), np.array(Vendors.longitude))
circles = {}
for v, color in zip(Vendors.status.unique(), inferno(Vendors.status.nunique())):
    tmp = Vendors[Vendors.status==v]
    if v == 'ACTIVE':
        color = 'Green'
    
    source = ColumnDataSource(tmp)
    circles[v] = p.circle(x=tmp['longitude'], y=tmp['latitude'], color=color, fill_alpha=0.7, radius=200, legend_label=v)
    circles[v].visible = False
    
p.legend.location = "top_left"
p.legend.click_policy="hide"


# output_file('Zooket_Vendor_Status.html')

show(p)

### Areas

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

query = """select Distinct Cities.id as 'City', Cities.title as `CityTitle`
            from zoodfood_db.Cities
            where id in {};""".format(tuple(areas.City.unique()))

cnx = mysql.connector.connect(user='mohammad_hossein', password='clwkehr2o8349h', host='db.snappfood.ir', port='3306')
cities = pd.read_sql(query, cnx)
cnx.close()

In [ ]:
gdf = gdf.merge(cities, on='City', how='left')

In [ ]:
gdf.head()

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("City", "@CityTitle"),
                    ("CityID", "@City"),
                   ("SuperArea", "@SuperArea"),
                    ("AreaTitle_English", "@AreaTitle_English"),
                    ("AreaTitle", "@AreaTitle"),
                    ("AreaID", "@AreaID"),])
p.add_tile(tile_provider)

p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.5, color = 'grey',
          line_color="black", line_width=0.3)

# output_file('Iran_Areas.html')

show(p)

### Canbo Tehran hyperstar

In [ ]:
import mysql.connector

query = """select V.id as 'StoreID', V.title, V.latitude, V.longitude
        from zoodfood_db.Vendors V
            join zoodfood_db.VendorTypes VT on vendor_type_id = VT.id
        where vendor_super_type_id = 4
--             and V.title Like '%هایپراستار%'
             and V.title Like '%جانبو%'
--             and V.title Like N'%اکسپرس مارکت%'
            and V.status <> 'SUSPEND';"""

cnx = mysql.connector.connect(user='mohammad_hossein', password='clwkehr2o8349h', host='db.snappfood.ir', port='3306')
hyperstar = pd.read_sql(query, cnx)
cnx.close()

In [ ]:
hyperstar.head()

In [ ]:
canbo.shape

In [ ]:
hyperfamily = pd.read_excel('canbo_hyperstar/Hyperfamili lat.long (1).xlsx')

In [ ]:
hyperfamily[['latitude', 'longitude']] = hyperfamily['latitude'].str.split(',', expand=True)

In [ ]:
hyperfamily.head()

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("FoodArea", "@AreaTitle"),
                    ("StoreID", "@StoreID"),
                    ("Company", "@Comp"),])
p.add_tile(tile_provider)

# ########### ZOOKET
# longitude, latitude = merc_from_arrays(np.array(Vendors.latitude), np.array(Vendors.longitude))

# source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(Vendors['id']), Comp=np.repeat('Zooket', len(latitude))))
# p.scatter(x='longitude', y='latitude', radius=100, fill_color='#FFFF00', fill_alpha=0.9, source=source)

########### HyperStar
longitude, latitude = merc_from_arrays(np.array(hyperstar.latitude.astype(float)), np.array(hyperstar.longitude.astype(float)))

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(hyperstar['StoreID']), Comp=np.repeat('HyperStar', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=100, fill_color='#0000FF', fill_alpha=0.9, source=source)

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(hyperstar['StoreID']), Comp=np.repeat('HyperStar', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=3500, fill_color='#0000FF', fill_alpha=0.6, source=source)

########### CANBO
longitude, latitude = merc_from_arrays(np.array(canbo.latitude), np.array(canbo.longitude))

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(canbo.StoreID), Comp=np.repeat('Canbo', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=100, fill_color='#83206B', fill_alpha=0.9, source=source)

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(canbo.StoreID), Comp=np.repeat('Canbo', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=2700, fill_color='#83206B', fill_alpha=0.6, source=source)

########### HyperStar
longitude, latitude = merc_from_arrays(np.array(hyperfamily.latitude.astype(float)), np.array(hyperfamily.longitude.astype(float)))

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(hyperfamily['StoreID']), Comp=np.repeat('HyperFamily', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=100, fill_color='#FFFF00', fill_alpha=0.9, source=source)

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(hyperfamily['StoreID']), Comp=np.repeat('HyperFamily', len(latitude))))
p.scatter(x='longitude', y='latitude', radius=3500, fill_color='#FFFF00', fill_alpha=0.6, source=source)


########### FOODAREAS
p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.3, color = 'grey',
          line_color="black", line_width=0.2)

output_file('Canbo_hyperstar_hyperfamily.html')

show(p)

### Zooket Vendors

In [ ]:
vendors = pd.read_excel('vendor (tehran).xlsx')

In [ ]:
vendors.shape

In [ ]:
import pymssql

query = """select id as 'VendorID', Title as 'VendorTitle', MarketingAreaName as 'Area', longitude, latitude
            from marketing_db.dbo.Vendors K
            where K.SuperTypeID=4
                and K.ID in {};""".format(str(tuple(vendors.VendorID)))

conn = pymssql.connect(server='warehouse.snappfood.ir:1410', user='zooket', password='?t!ytN9JW39<3[zj', database='marketing_db')
V = pd.read_sql(query, conn)
conn.close()

In [ ]:
V.head()

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("VendorID", "@VendorID"),
                    ("VendorTitle", "@VendorTitle"),
                    ("Area", "@Area"),])
p.add_tile(tile_provider)

########### ZOOKET
longitude, latitude = merc_from_arrays(np.array(V.latitude), np.array(V.longitude))

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, VendorID=np.array(V['VendorID']), VendorTitle=np.array(V['VendorTitle']), Area=np.array(V['Area'])))
p.scatter(x='longitude', y='latitude', radius=200, fill_color='#FFAF00', fill_alpha=0.9, source=source)

output_file('Zooket_selected_vendors.html')

show(p)

## Chain

In [ ]:
from shapely import wkt

areas = pd.read_excel('x220312_v5.xlsx', sheet_name='Area')

areas = areas.pipe(gpd.GeoDataFrame)
areas.columns = ['area_name', 'geometry', 'sum_value', 'color']

areas['geometry'] = areas['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(areas)

gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(epsg=3857)

In [ ]:
vendors = pd.read_excel('x220312_v5.xlsx', sheet_name='Vendor')

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[("area_name", "@area_name"),
                     ("sum_value", "@sum_value"),
                    ("VendorName", "@VendorName")])
p.add_tile(tile_provider)

p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.5, color = 'color',
          line_color="black", line_width=0.2)

longitude, latitude = merc_from_arrays(np.array(vendors['Lat'].astype(float)), np.array(vendors['Long'].astype(float)))
source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude,
                                    VendorName=np.array(vendors['Flag Name']),
                                   color=np.array(vendors['color'])))

p.scatter(x='longitude', y='latitude', radius=200, fill_alpha=0.9, fill_color='color', source=source)
# p.scatter(x='longitude', y='latitude', radius=500, fill_alpha=0.3,fill_color='black', source=source)

output_file('Chain_map.html')

show(p)

In [ ]:
vendors.head()